In [82]:
import pandas as pd
#import gspread
import requests 
import io
import matplotlib.pyplot as plt
import scipy
import numpy as np
import utils
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

Part 2

In [83]:
# AL Train Data Set
ALdata = pd.read_csv('ALtrain', low_memory=False, lineterminator ='\n', delimiter = ' ', header = None, names = ['Word', 'Tag'])
ALdata.head(15)

,Word,Tag
0,小,B-DISTRICT
1,港,I-DISTRICT
2,四,B-POI
3,方,I-POI
4,家,I-POI
5,园,I-POI
6,17,B-HOUSENO
7,幢,I-HOUSENO
8,1363,B-CELLNO
9,室,I-CELLNO


In [84]:
# CN Train Data Set
CNdata = pd.read_csv('CNtrain', low_memory=False, lineterminator ='\n', delimiter = ' ', header = None, names = ['Word', 'Tag'])
CNdata.head(15)

,Word,Tag
0,这,O
1,大概,O
2,是,O
3,所有,O
4,版本,O
5,里,O
6,无忌,B-neutral
7,哥哥,I-neutral
8,对,O
9,敏敏,B-neutral


In [85]:
# EN Train Data Set
ENdata = pd.read_csv('ENtrain', low_memory=False, lineterminator ='\n', delimiter = ' ', header = None, names = ['Word', 'Tag'])
ENdata.head(15)

,Word,Tag
0,Municipal,B-NP
1,bonds,I-NP
2,are,B-VP
3,generally,B-ADVP
4,a,B-ADJP
5,bit,I-ADJP
6,safer,I-ADJP
7,than,B-PP
8,corporate,B-NP
9,bonds,I-NP


In [86]:
# SG Train Data Set
SGdata = pd.read_csv('SGtrain', low_memory=False, lineterminator ='\n', delimiter = ' ', header = None, names = ['Word', 'Tag'])
SGdata.head(15)

,Word,Tag
0,I'm,O
1,about,O
2,to,O
3,see,O
4,Adam,B-positive
5,Levine,I-positive
6,and,O
7,I'm,O
8,probably,O
9,going,O


In [87]:
print('EN:')
print(ENdata['Tag'].value_counts())
print('\n')

print('AL:')
print(ALdata['Tag'].value_counts())
print('\n')

print('CN:')
print(CNdata['Tag'].value_counts())
print('\n')

print('SG:')
print(SGdata['Tag'].value_counts())
print('\n')

EN:
I-NP       54591
B-NP       47305
O          23872
B-PP       18387
B-VP       18261
I-VP       10159
B-ADVP      3565
B-SBAR      1899
B-ADJP      1751
I-ADJP       574
B-PRT        468
I-ADVP       363
I-PP         223
I-CONJP       64
B-CONJP       49
I-SBAR        48
B-INTJ        26
B-LST         11
I-INTJ         7
I-UCP          4
B-UCP          1
Name: Tag, dtype: int64


AL:
I-POI          28472
I-ROAD         15178
I-DISTRICT     13364
I-TOWN         10748
I-CITY         10430
I-PROV          8666
B-POI           7348
B-DISTRICT      6856
B-ROAD          6324
B-CITY          5697
B-ROADNO        5061
I-ROADNO        4666
B-TOWN          4612
I-SUBPOI        4557
B-PROV          4488
B-REDUNDANT     4137
I-REDUNDANT     3668
B-HOUSENO       3455
I-COMMUNITY     3148
B-ROOMNO        3107
I-HOUSENO       3045
I-PERSON        2232
I-DEVZONE       1834
B-SUBPOI        1650
I-CELLNO        1488
B-COMMUNITY     1479
B-CELLNO        1326
I-FLOORNO       1237
B-FLOORNO       1229


In [88]:
# Write a function that estimates the emission parameters from the training set using MLE (maximum likelihood estimation)
# e(x|y) = Count (y -> x)/Count(y)
 
def emissionEstimate(data, word, tag):
    tagcheck = (data.Tag == tag)
    y = tagcheck.sum()
    xy = (data.Word[tagcheck] == word).sum()
    
    return xy/y

In [89]:
# replace the words that appear less than k times in the training set with
# special word token #UNK# before training. This leads to a "modified training set"
# if a word does not appear in the "modified training set", we replace that with #UNK# as well
# change k to whatever value you want

#Smoothing of EN
#k = 3
EN = pd.DataFrame(ENdata['Word'].value_counts()< 3)
EN.apply(lambda val: val == True)
k1 = EN.index[EN['Word'] == True].tolist()
ENdata.loc[ENdata['Word'].isin(k1), 'Word'] = '#UNK#'
ENdata.head(20)

,Word,Tag
0,#UNK#,B-NP
1,bonds,I-NP
2,are,B-VP
3,generally,B-ADVP
4,a,B-ADJP
5,bit,I-ADJP
6,#UNK#,I-ADJP
7,than,B-PP
8,corporate,B-NP
9,bonds,I-NP


In [91]:
#Smoothing of AL
AL = pd.DataFrame(ALdata['Word'].value_counts()<3)
AL.apply(lambda val: val == True)
k2 = AL.index[AL['Word'] == True].tolist()
ALdata.loc[ALdata['Word'].isin(k2), 'Word'] = '#UNK#'
ALdata.head(20)

,Word,Tag
0,小,B-DISTRICT
1,港,I-DISTRICT
2,四,B-POI
3,方,I-POI
4,家,I-POI
5,园,I-POI
6,17,B-HOUSENO
7,幢,I-HOUSENO
8,1363,B-CELLNO
9,室,I-CELLNO


In [92]:
#Smoothing of CN
CN = pd.DataFrame(CNdata['Word'].value_counts()<3)
CN.apply(lambda val: val == True)
k3 = CN.index[CN['Word'] == True].tolist()
CNdata.loc[CNdata['Word'].isin(k3), 'Word'] = '#UNK#'
CNdata.head(20)

,Word,Tag
0,这,O
1,大概,O
2,是,O
3,所有,O
4,版本,O
5,里,O
6,#UNK#,B-neutral
7,哥哥,I-neutral
8,对,O
9,#UNK#,B-neutral


In [93]:
#Smoothing of SG
SG = pd.DataFrame(SGdata['Word'].value_counts()<3)
SG.apply(lambda val: val == True)
k4 = SG.index[SG['Word'] == True].tolist()
SGdata.loc[SGdata['Word'].isin(k4), 'Word'] = '#UNK#'
SGdata.head(20)

,Word,Tag
0,I'm,O
1,about,O
2,to,O
3,see,O
4,Adam,B-positive
5,Levine,I-positive
6,and,O
7,I'm,O
8,probably,O
9,going,O


Goal: Obtain a dataset with Word, Tag, Emission

Steps
1) Obtain count of tags per word
2) Find the Count of the different tags across the whole dataset
3) For each tag for each word, find out what the total count for each tag
4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
5) Keep only the word/tag with highest emission value

Finally, export pdf

In [251]:
# Goal: Obtain a dataset with Word, Tag, Emission

# Steps
# 1) Obtain count of tags per word
# 2) Find the Count of the different tags across the whole dataset
# 3) For each tag for each word, find out what the total count for each tag
# 4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
# 5) Keep only the word/tag with highest emission value

#EN

# 1)
ENWordTagcount = pd.DataFrame({'Count' : ENdata.groupby(['Word', 'Tag']).size()}).reset_index()
ENWordTagcount

0             O
1        B-ADJP
2          B-NP
3          I-NP
4             O
          ...  
12166      I-NP
12167      B-NP
12168      I-NP
12169      B-NP
12170      I-NP
Name: Tag, Length: 12171, dtype: object

In [275]:
# 2)
ENTagcount = pd.DataFrame({'Global Count' :ENdata['Tag'].value_counts()}).reset_index()
ENTagcount.rename(columns = {'index' : 'Tag'}, inplace= True)

ENTagcount

,Tag,Global Count
0,I-NP,54591
1,B-NP,47305
2,O,23872
3,B-PP,18387
4,B-VP,18261
5,I-VP,10159
6,B-ADVP,3565
7,B-SBAR,1899
8,B-ADJP,1751
9,I-ADJP,574


In [277]:
# 3
ENTotalTagCount = pd.merge(ENWordTagcount, ENTagcount, on='Tag')
ENTotalTagCount

,Word,Tag,Count,Global Count
0,!,O,15,23872
1,#,O,1,23872
2,#UNK#,O,56,23872
3,$,O,2,23872
4,&,O,3,23872
...,...,...,...,...
12166,before,I-SBAR,1,48
12167,if,I-SBAR,15,48
12168,that,I-SBAR,9,48
12169,though,I-SBAR,15,48


In [279]:
# 4)
ENTotalTagCount['Emission'] = ENTotalTagCount['Count']/ENTotalTagCount['Global Count']
ENTotalTagCount

,Word,Tag,Count,Global Count,Emission
0,!,O,15,23872,0.000628
1,#,O,1,23872,0.000042
2,#UNK#,O,56,23872,0.002346
3,$,O,2,23872,0.000084
4,&,O,3,23872,0.000126
...,...,...,...,...,...
12166,before,I-SBAR,1,48,0.020833
12167,if,I-SBAR,15,48,0.312500
12168,that,I-SBAR,9,48,0.187500
12169,though,I-SBAR,15,48,0.312500


In [289]:
# 5)
idx = ENTotalTagCount.groupby(['Word'])['Emission'].transform(max) == ENTotalTagCount['Emission']
ENmodel = ENTotalTagCount[idx]

,Word,Tag,Count,Global Count,Emission
0,!,O,15,23872,0.000628
6,'',O,1168,23872,0.048928
8,",",O,8705,23872,0.364653
9,-,O,11,23872,0.000461
10,--,O,358,23872,0.014997
...,...,...,...,...,...
12164,well,I-CONJP,18,64,0.281250
12166,before,I-SBAR,1,48,0.020833
12167,if,I-SBAR,15,48,0.312500
12169,though,I-SBAR,15,48,0.312500


In [294]:
# export to CSV
ENmodel1 = ENmodel[['Word', 'Tag']]
file = "ENdev.p2.out"
ENmodel1.to_csv(file, sep=' ', index=False, header = False)

In [295]:
# Goal: Obtain a dataset with Word, Tag, Emission

# Steps
# 1) Obtain count of tags per word
# 2) Find the Count of the different tags across the whole dataset
# 3) For each tag for each word, find out what the total count for each tag
# 4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
# 5) Keep only the word/tag with highest emission value

#AL

# 1)
ALWordTagcount = pd.DataFrame({'Count' : ALdata.groupby(['Word', 'Tag']).size()}).reset_index()
ALWordTagcount

,Word,Tag,Count
0,#UNK#,B-ASSIST,4
1,#UNK#,B-CELLNO,5
2,#UNK#,B-CITY,18
3,#UNK#,B-COMMUNITY,47
4,#UNK#,B-DEVZONE,2
...,...,...,...
13774,龙,I-SUBROAD,2
13775,龙,I-TOWN,27
13776,龚,B-POI,5
13777,龚,I-POI,1


In [296]:
# 2)
ALTagcount = pd.DataFrame({'Global Count' :ALdata['Tag'].value_counts()}).reset_index()
ALTagcount.rename(columns = {'index' : 'Tag'}, inplace= True)

ALTagcount

,Tag,Global Count
0,I-POI,28472
1,I-ROAD,15178
2,I-DISTRICT,13364
3,I-TOWN,10748
4,I-CITY,10430
5,I-PROV,8666
6,B-POI,7348
7,B-DISTRICT,6856
8,B-ROAD,6324
9,B-CITY,5697


In [297]:
# 3
ALTotalTagCount = pd.merge(ALWordTagcount, ALTagcount, on='Tag')
ALTotalTagCount

,Word,Tag,Count,Global Count
0,#UNK#,B-ASSIST,4,801
1,-,B-ASSIST,2,801
2,1,B-ASSIST,1,801
3,150,B-ASSIST,1,801
4,2,B-ASSIST,3,801
...,...,...,...,...
13774,中,B-COUNTRY,47,47
13775,国,I-COUNTRY,47,47
13776,电,B-OTHERINFO,1,3
13777,转,B-OTHERINFO,1,3


In [298]:
# 4)
ALTotalTagCount['Emission'] = ALTotalTagCount['Count']/ALTotalTagCount['Global Count']
ALTotalTagCount

,Word,Tag,Count,Global Count,Emission
0,#UNK#,B-ASSIST,4,801,0.004994
1,-,B-ASSIST,2,801,0.002497
2,1,B-ASSIST,1,801,0.001248
3,150,B-ASSIST,1,801,0.001248
4,2,B-ASSIST,3,801,0.003745
...,...,...,...,...,...
13774,中,B-COUNTRY,47,47,1.000000
13775,国,I-COUNTRY,47,47,1.000000
13776,电,B-OTHERINFO,1,3,0.333333
13777,转,B-OTHERINFO,1,3,0.333333


In [300]:
# 5)
idx = ALTotalTagCount.groupby(['Word'])['Emission'].transform(max) == ALTotalTagCount['Emission']
ALmodel = ALTotalTagCount[idx]
ALmodel

,Word,Tag,Count,Global Count,Emission
6,400,B-ASSIST,1,801,0.001248
11,d,B-ASSIST,1,801,0.001248
18,与,B-ASSIST,79,801,0.098627
19,东,B-ASSIST,44,801,0.054931
21,之,B-ASSIST,2,801,0.002497
...,...,...,...,...,...
13774,中,B-COUNTRY,47,47,1.000000
13775,国,I-COUNTRY,47,47,1.000000
13776,电,B-OTHERINFO,1,3,0.333333
13777,转,B-OTHERINFO,1,3,0.333333


In [301]:
# export to CSV
ALmodel1 = ALmodel[['Word', 'Tag']]
file = "ALdev.p2.out"
ALmodel1.to_csv(file, sep=' ', index=False, header = False)

In [302]:
# Goal: Obtain a dataset with Word, Tag, Emission

# Steps
# 1) Obtain count of tags per word
# 2) Find the Count of the different tags across the whole dataset
# 3) For each tag for each word, find out what the total count for each tag
# 4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
# 5) Keep only the word/tag with highest emission value

#CN

# 1)
CNWordTagcount = pd.DataFrame({'Count' : CNdata.groupby(['Word', 'Tag']).size()}).reset_index()
CNWordTagcount

,Word,Tag,Count
0,!,I-neutral,4
1,!,I-positive,4
2,!,O,68
3,#,B-negative,11
4,#,B-neutral,453
...,...,...,...
10781,？,I-neutral,3
10782,？,O,321
10783,～,O,67
10784,￣,O,6


In [303]:
# 2)
CNTagcount = pd.DataFrame({'Global Count' :CNdata['Tag'].value_counts()}).reset_index()
CNTagcount.rename(columns = {'index' : 'Tag'}, inplace= True)

CNTagcount

,Tag,Global Count
0,O,123304
1,I-neutral,6030
2,B-neutral,5252
3,I-positive,3845
4,B-positive,3087
5,B-negative,449
6,I-negative,275


In [304]:
# 3
CNTotalTagCount = pd.merge(CNWordTagcount, CNTagcount, on='Tag')
CNTotalTagCount

,Word,Tag,Count,Global Count
0,!,I-neutral,4,6030
1,#,I-neutral,5,6030
2,#UNK#,I-neutral,1501,6030
3,&,I-neutral,4,6030
4,',I-neutral,7,6030
...,...,...,...,...
10781,马桶,I-negative,1,275
10782,马蓉,I-negative,1,275
10783,鹿晗,I-negative,1,275
10784,麻雀,I-negative,1,275


In [305]:
# 4)
CNTotalTagCount['Emission'] = CNTotalTagCount['Count']/CNTotalTagCount['Global Count']
CNTotalTagCount

,Word,Tag,Count,Global Count,Emission
0,!,I-neutral,4,6030,0.000663
1,#,I-neutral,5,6030,0.000829
2,#UNK#,I-neutral,1501,6030,0.248922
3,&,I-neutral,4,6030,0.000663
4,',I-neutral,7,6030,0.001161
...,...,...,...,...,...
10781,马桶,I-negative,1,275,0.003636
10782,马蓉,I-negative,1,275,0.003636
10783,鹿晗,I-negative,1,275,0.003636
10784,麻雀,I-negative,1,275,0.003636


In [306]:
# 5)
idx = CNTotalTagCount.groupby(['Word'])['Emission'].transform(max) == CNTotalTagCount['Emission']
CNmodel = CNTotalTagCount[idx]
CNmodel

,Word,Tag,Count,Global Count,Emission
3,&,I-neutral,4,6030,0.000663
4,',I-neutral,7,6030,0.001161
5,(,I-neutral,12,6030,0.001990
6,),I-neutral,11,6030,0.001824
12,0,I-neutral,2,6030,0.000332
...,...,...,...,...,...
10779,食品,I-negative,2,275,0.007273
10780,香菇,I-negative,2,275,0.007273
10781,马桶,I-negative,1,275,0.003636
10783,鹿晗,I-negative,1,275,0.003636


In [307]:
# export to CSV
CNmodel1 = CNmodel[['Word', 'Tag']]
file = "CNdev.p2.out"
CNmodel1.to_csv(file, sep=' ', index=False, header = False)

In [309]:
# Goal: Obtain a dataset with Word, Tag, Emission

# Steps
# 1) Obtain count of tags per word
# 2) Find the Count of the different tags across the whole dataset
# 3) For each tag for each word, find out what the total count for each tag
# 4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
# 5) Keep only the word/tag with highest emission value

#CN

# 1)
SGWordTagcount = pd.DataFrame({'Count' : SGdata.groupby(['Word', 'Tag']).size()}).reset_index()
SGWordTagcount

,Word,Tag,Count
0,O\n,O,23
1,!,I-negative,3
2,!,I-neutral,2
3,!,I-positive,2
4,!,O,2469
...,...,...,...
10587,➖,O,9
10588,⭐,O,5
10589,・,O,12
10590,️,I-positive,1


In [310]:
# 2)
SGTagcount = pd.DataFrame({'Global Count' :SGdata['Tag'].value_counts()}).reset_index()
SGTagcount.rename(columns = {'index' : 'Tag'}, inplace= True)

SGTagcount

,Tag,Global Count
0,O,133993
1,B-neutral,9736
2,I-neutral,9050
3,B-positive,3521
4,I-positive,2309
5,B-negative,1818
6,I-negative,708


In [311]:
# 3
SGTotalTagCount = pd.merge(SGWordTagcount, SGTagcount, on='Tag')
SGTotalTagCount

,Word,Tag,Count,Global Count
0,O\n,O,23,133993
1,!,O,2469,133993
2,#,O,20,133993
3,#Accounting,O,6,133993
4,#Albany,O,3,133993
...,...,...,...,...
10587,york,B-positive,1,3521
10588,you,B-positive,3,3521
10589,young,B-positive,1,3521
10590,your,B-positive,1,3521


In [312]:
# 4)
SGTotalTagCount['Emission'] = SGTotalTagCount['Count']/SGTotalTagCount['Global Count']
SGTotalTagCount

,Word,Tag,Count,Global Count,Emission
0,O\n,O,23,133993,0.000172
1,!,O,2469,133993,0.018426
2,#,O,20,133993,0.000149
3,#Accounting,O,6,133993,0.000045
4,#Albany,O,3,133993,0.000022
...,...,...,...,...,...
10587,york,B-positive,1,3521,0.000284
10588,you,B-positive,3,3521,0.000852
10589,young,B-positive,1,3521,0.000284
10590,your,B-positive,1,3521,0.000284


In [313]:
# 5)
idx = SGTotalTagCount.groupby(['Word'])['Emission'].transform(max) == SGTotalTagCount['Emission']
SGmodel = SGTotalTagCount[idx]
SGmodel

,Word,Tag,Count,Global Count,Emission
0,O\n,O,23,133993,0.000172
1,!,O,2469,133993,0.018426
2,#,O,20,133993,0.000149
3,#Accounting,O,6,133993,0.000045
6,#Art,O,4,133993,0.000030
...,...,...,...,...,...
10585,white,B-positive,3,3521,0.000852
10586,women's,B-positive,1,3521,0.000284
10587,york,B-positive,1,3521,0.000284
10589,young,B-positive,1,3521,0.000284


In [314]:
# export to CSV
SGmodel1 = SGmodel[['Word', 'Tag']]
file = "SGdev.p2.out"
SGmodel1.to_csv(file, sep=' ', index=False, header = False)

Evaluation Scripts

In [316]:
!python3 ./EvalScript/evalResult.py ./SG/dev.out ./SGdev.p2.out


#Entity in gold data: 4537
#Entity in prediction: 1366

#Correct Entity : 0
Entity  precision: 0.0000
Entity  recall: 0.0000
Entity  F: 0.0000

#Correct Sentiment : 0
Sentiment  precision: 0.0000
Sentiment  recall: 0.0000
Sentiment  F: 0.0000
